In [9]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GroupShuffleSplit

Parameters to tune: l1 and/or l2 reg

### Import Subsampled Data

In [3]:
os.chdir('../data')

X = pd.read_csv('X_3specialties_equalWeight_subsample.zip',compression='zip', index_col=False)
y = pd.read_csv('y_3specialties_equalWeight_subsample.zip',compression='zip')
groups = pd.read_csv('groups_3specialties_equalWeight_subsample.zip',compression='zip')

X = X.iloc[:,1:]
y = y.iloc[:,1:]
groups = groups.iloc[:,1:]

y_columns = y.columns

le = LabelEncoder()
y = y.values.ravel()
y = le.fit_transform(y)
y = pd.DataFrame(y)
y.columns = y_columns

NameError: name 'LabelEncoder' is not defined

In [ ]:
X.shape

In [12]:
categorical_ftrs = ['Prscrbr_City',
                    'Prscrbr_State_Abrvtn',
                    'Brnd_Name',
                    'Gnrc_Name']

std_ftrs = ['Tot_Clms', 
            'Tot_30day_Fills', 
            'Tot_Day_Suply', 
            'Tot_Drug_Cst', 
            'Tot_Benes', 
            'GE65_Tot_Clms',
            'GE65_Tot_30day_Fills',
            'GE65_Tot_Drug_Cst',
            'GE65_Tot_Day_Suply',
            'GE65_Tot_Benes']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'), categorical_ftrs),
        ('std', StandardScaler(), std_ftrs)])


##### Imputation

In [36]:
X_subsample2.iloc[32]

Prscrbr_City              Yuba City
Prscrbr_State_Abrvtn             CA
Brnd_Name               Allopurinol
Gnrc_Name               Allopurinol
Tot_Clms                         35
Tot_30day_Fills                87.0
Tot_Day_Suply                  2610
Tot_Drug_Cst                 505.25
Tot_Benes                       NaN
GE65_Tot_Clms                   NaN
GE65_Tot_30day_Fills            NaN
GE65_Tot_Drug_Cst               NaN
GE65_Tot_Day_Suply              NaN
GE65_Tot_Benes                  NaN
Name: 32, dtype: object

In [4]:
param_grid1 = {'solver': ['saga'],
    'penalty' : ['none']}   #'solver'  : ['newton-cg', 'lbfgs', 'liblinear']


param_grid2 = {'solver': ['saga'],
               'penalty' : ['l1','l2'], 
               'C'       : np.logspace(-3,3,7)}   #'solver'  : ['newton-cg', 'lbfgs', 'liblinear']

In [54]:
np.logspace(-2,2,5)

array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])

### Imputer w/ Linear Regression

In [5]:
"""
%%time

## Perform 60-20-20 Split
# splitter for subsampled data
stratGroupKFold2 = StratifiedGroupKFold(n_splits=5)

# splitter for other
stratGroupKFold3 = StratifiedGroupKFold(n_splits=4)


nr_states = [0] #,1,2
counter = 0
counter2 = 0
final_models = []


for i_other2,i_test2 in stratGroupKFold2.split(X_subsample2.values, y_subsample2, groups_subsample2.values):
    
    counter = counter + 1
    
    X_other2, y_other2, groups_other2 = X_subsample2.iloc[i_other2], y_subsample2.iloc[i_other2], groups_subsample2.iloc[i_other2]
    X_test2, y_test2, groups_test2 = X_subsample2.iloc[i_test2], y_subsample2.iloc[i_test2], groups_subsample2.iloc[i_test2]
    
    
    print(f'Test Set #{counter}')

    print("    Test Set Size:", len(y_test2))

    print()
    
    #for i in range(len(nr_states)):

        #print("         Random State:", i)
        #print()
        
    counter2 = 0
    
    for i in range(len(nr_states)):
    
        for i_train3,i_val3 in stratGroupKFold3.split(X_other2.values, y_other2.values, groups_other2.values):
            counter2 = counter2 + 1

            # Perform n-Fold CV
            #cv = stratGroupKFold3.split(X_other2, y_other2, groups_other2)

            X_train, y_train, groups_train = X_other2.iloc[i_train3], y_other2.iloc[i_train3], groups_other2.iloc[i_train3]
            X_val, y_val, groups_val = X_other2.iloc[i_val3], y_other2.iloc[i_val3], groups_other2.iloc[i_val3]

            print(f"    Train Set #{counter2} Size: {len(X_train)}")
            print()


            X_prep = preprocessor.fit_transform(X_train)

            # collect feature names
            feature_names = preprocessor.get_feature_names_out()

            df_train = pd.DataFrame(data=X_prep,columns=feature_names)
            print(f"    Train Set Shape after preprocessing: {df_train.shape}")
            print()

            # transform the CV
            df_CV = preprocessor.transform(X_val)
            df_CV = pd.DataFrame(data=df_CV,columns = feature_names)
            print(f"    Validation Set Shape after preprocessing: {df_CV.shape}")
            print()


            # transform the test
            df_test = preprocessor.transform(X_test2)
            df_test = pd.DataFrame(data=df_test,columns = feature_names)
            print(f"    Test Set Shape after preprocessing: {df_test.shape}")
            print()


            ### Multivariate Imputation
            imputer = IterativeImputer(estimator = LinearRegression(), random_state=i)
            X_impute = imputer.fit_transform(df_train)
            df_train_imp = pd.DataFrame(data=X_impute, columns = df_train.columns)


            df_CV_imp = pd.DataFrame(data=imputer.transform(df_CV), columns = df_train.columns)
            df_test_imp = pd.DataFrame(data=imputer.transform(df_test), columns = df_train.columns)
            
            print('Imputation Step ----> Done')


            ### Logistic Regression
            #logreg = LogisticRegression()

            #grid = GridSearchCV(logreg,                    # model
                       #param_grid = parameters,   # hyperparameters
                       #scoring='accuracy')

            #grid.fit(df_CV_imp, y_val, groups = groups_val, )


            train_score = np.zeros(len(ParameterGrid(param_grid)))
            val_score = np.zeros(len(ParameterGrid(param_grid)))
            models = []

            # loop through all combinations of hyperparameter combos
            for p in range(len(ParameterGrid(param_grid))):
                params = ParameterGrid(param_grid1)[p]
                print('   ',params) 
                clf = LogisticRegression(**params, random_state = i, n_jobs=1) # initialize the classifier
                clf.fit(df_train_imp,y_train) # fit the model
                models.append(clf) # save it
                # calculate train and validation accuracy scores
                y_train_pred = clf.predict(df_train_imp)
                train_score[p] = accuracy_score(y_train,y_train_pred)
                y_val_pred = clf.predict(df_CV_imp)
                val_score[p] = accuracy_score(y_val,y_val_pred)
                print('   ',train_score[p],val_score[p])

            # print out model parameters that maximize validation accuracy
            print('best model parameters:',ParameterGrid(param_grid)[np.argmax(val_score)])
            print('corresponding validation score:',np.max(val_score))
            # collect and save the best model
            final_models.append(models[np.argmax(val_score)])
            # calculate and save the test score
            #y_test_pred = final_models[-1].predict(X_test_prep)
            #test_scores[i] = accuracy_score(y_test,y_test_pred)
            #print('test score:',test_scores[i])
"""

'\n%%time\n\n## Perform 60-20-20 Split\n# splitter for subsampled data\nstratGroupKFold2 = StratifiedGroupKFold(n_splits=5)\n\n# splitter for other\nstratGroupKFold3 = StratifiedGroupKFold(n_splits=4)\n\n\nnr_states = [0] #,1,2\ncounter = 0\ncounter2 = 0\nfinal_models = []\n\n\nfor i_other2,i_test2 in stratGroupKFold2.split(X_subsample2.values, y_subsample2, groups_subsample2.values):\n    \n    counter = counter + 1\n    \n    X_other2, y_other2, groups_other2 = X_subsample2.iloc[i_other2], y_subsample2.iloc[i_other2], groups_subsample2.iloc[i_other2]\n    X_test2, y_test2, groups_test2 = X_subsample2.iloc[i_test2], y_subsample2.iloc[i_test2], groups_subsample2.iloc[i_test2]\n    \n    \n    print(f\'Test Set #{counter}\')\n\n    print("    Test Set Size:", len(y_test2))\n\n    print()\n    \n    #for i in range(len(nr_states)):\n\n        #print("         Random State:", i)\n        #print()\n        \n    counter2 = 0\n    \n    for i in range(len(nr_states)):\n    \n        for i_t

In [10]:
def ML_pipeline_ImputationLogisticReg_GridSearchCV(X, y, groups, i):
    # create a test set based on groups
    splitter = GroupShuffleSplit(n_splits=1,test_size=0.2,random_state=i)
    
    # Get Test Set
    for i_other,i_test in splitter.split(X, y, groups):
        X_other, y_other, groups_other = X.iloc[i_other], y.iloc[i_other], groups.iloc[i_other]
        X_test, y_test, groups_test = X.iloc[i_test], y.iloc[i_test], groups.iloc[i_test]
    
    # Get Validation Set
    
    kf = GroupKFold(n_splits=5)
    counter = 0
    for i_train, i_test in kf.split(X_other, y_other, groups_other):
        X_train, y_train, groups_train = X_other.iloc[i_train], y_other.iloc[i_train], groups_other.iloc[i_train]
        X_val, y_val, groups_val = X_other.iloc[i_test], y_other.iloc[i_test], groups_other.iloc[i_test]
        
        #print(len(y_val))
        #print(len(y_train))
        counter = counter + 1
        
        print(f"CV # {counter}")
        X_prep = preprocessor.fit_transform(X_train)
        # collect feature names
        
        feature_names = preprocessor.get_feature_names_out()
        
        df_train = pd.DataFrame(data=X_prep,columns=feature_names)
        print(f"Train Set Shape (after preprocessing): {df_train.shape}")
        print()
        
        # transform the CV
        df_CV = preprocessor.transform(X_val)
        df_CV = pd.DataFrame(data=df_CV,columns = feature_names)
        print(f"CV Set Shape (after preprocessing): {df_CV.shape}")
        print()
        # transform the test
        df_test = preprocessor.transform(X_test)
        df_test = pd.DataFrame(data=df_test,columns = feature_names)
        print(f"Test Set Shape (after preprocessing): {df_test.shape}")
        print()
        
        y_CV = y_val
        
        
        ### Multivariate Imputation
        imputer = IterativeImputer(estimator = LinearRegression(), random_state=i)
        X_impute = imputer.fit_transform(df_train)
        df_train_imp = pd.DataFrame(data=X_impute, columns = df_train.columns)


        df_CV_imp = pd.DataFrame(data=imputer.transform(df_CV), columns = df_train.columns)
        df_test_imp = pd.DataFrame(data=imputer.transform(df_test), columns = df_train.columns)

        train_score = np.zeros(len(ParameterGrid(param_grid)))
        val_score = np.zeros(len(ParameterGrid(param_grid)))
        models = []
        
        print("*** Imputation Done ***")
        print()

        # loop through all combinations of hyperparameter combos
        for p in range(len(ParameterGrid(param_grid1))):
            params = ParameterGrid(param_grid1)[p]
            print('   ',params) 
            clf = LogisticRegression(**params, random_state = i, n_jobs=1) # initialize the classifier
            clf.fit(df_train_imp,y_train) # fit the model
            models.append(clf) # save it
            # calculate train and validation accuracy scores
            y_train_pred = clf.predict(df_train_imp)
            train_score[p] = f1_score(y_train,y_train_pred, average = 'macro')
            y_val_pred = clf.predict(df_CV_imp)
            val_score[p] = f1_score(y_val,y_val_pred, average = 'macro')
            print('   ',train_score[p],val_score[p])

        # print out model parameters that maximize validation accuracy
        print('best model parameters:',ParameterGrid(param_grid)[np.argmax(val_score)])
        print('corresponding validation score:',np.max(val_score))
        # collect and save the best model
        final_models.append(models[np.argmax(val_score)])
        # calculate and save the test score
        y_test_pred = final_models[-1].predict(X_test_prep)
        acc = accuracy_score(y_test,y_test_pred)
        test_score = f1_score(y_test,y_test_pred, average = 'macro')
        print('f1 score:',test_scores)
        print('accuracy score:',acc)
        #total_accuracy, f1, cm = reduced_feature_xgb(df_train, y_train, df_CV, y_CV, df_test, y_test, i)
        cm = confusion_matrix(y_test,y_test_pred)
        return test_score, acc, cm

In [13]:
%%time
final_models = []
test_scores = []
best_params = []
confusion_mat = []
class_met = []
accuracy_scores = []

for i in range(1):
    print(f'Random State # {i}')
    print()
    
    test_score, acc, cm = ML_pipeline_ImputationLogisticReg_GridSearchCV(X, y, groups, i)
    
    confusion_mat.append(cm)
    
    #class_met.append(class_metrics)
    
    print()
    print('test score:', test_score)
    test_scores.append(test_score)
    print()
    accuracy_scores.append(acc)
    
print('test accuracy:',np.around(np.mean(test_scores),2),'+/-',np.around(np.std(test_scores),2))

Random State # 0

CV # 1
Train Set Shape (after preprocessing): (2808, 856)

CV Set Shape (after preprocessing): (702, 856)

Test Set Shape (after preprocessing): (694, 856)



KeyboardInterrupt: 

### Imputer w/ RandomForestRegressor

In [ ]:
"""
imputer = IterativeImputer(estimator = RandomForestRegressor(n_estimators=1), random_state=42)


%%time

## Perform 60-20-20 Split
# splitter for subsampled data
stratGroupKFold2 = StratifiedGroupKFold(n_splits=5)

# splitter for other
stratGroupKFold3 = StratifiedGroupKFold(n_splits=4)


nr_states = [0] #,1,2
counter = 0
counter2 = 0
final_models = []


for i_other2,i_test2 in stratGroupKFold2.split(X_subsample2.values, y_subsample2, groups_subsample2.values):
    
    counter = counter + 1
    
    X_other2, y_other2, groups_other2 = X_subsample2.iloc[i_other2], y_subsample2.iloc[i_other2], groups_subsample2.iloc[i_other2]
    X_test2, y_test2, groups_test2 = X_subsample2.iloc[i_test2], y_subsample2.iloc[i_test2], groups_subsample2.iloc[i_test2]
    
    
    print(f'Test Set #{counter}')

    print("    Test Set Size:", len(y_test2))

    print()
    
    #for i in range(len(nr_states)):

        #print("         Random State:", i)
        #print()
        
    counter2 = 0
    
    for i in range(len(nr_states)):
    
        for i_train3,i_val3 in stratGroupKFold3.split(X_other2.values, y_other2.values, groups_other2.values):
            counter2 = counter2 + 1

            # Perform n-Fold CV
            #cv = stratGroupKFold3.split(X_other2, y_other2, groups_other2)

            X_train, y_train, groups_train = X_subsample2.iloc[i_train3], y_subsample2.iloc[i_train3], groups_subsample2.iloc[i_train3]
            X_val, y_val, groups_val = X_subsample2.iloc[i_val3], y_subsample2.iloc[i_val3], groups_subsample2.iloc[i_val3]

            print(f"    Train Set #{counter2} Size: {len(X_train)}")
            print()


            X_prep = preprocessor.fit_transform(X_train)

            # collect feature names
            feature_names = preprocessor.get_feature_names_out()

            df_train = pd.DataFrame(data=X_prep,columns=feature_names)
            print(f"    Train Set Shape after preprocessing: {df_train.shape}")
            print()

            # transform the CV
            df_CV = preprocessor.transform(X_val)
            df_CV = pd.DataFrame(data=df_CV,columns = feature_names)
            print(f"    Validation Set Shape after preprocessing: {df_CV.shape}")
            print()


            # transform the test
            df_test = preprocessor.transform(X_test2)
            df_test = pd.DataFrame(data=df_test,columns = feature_names)
            print(f"    Test Set Shape after preprocessing: {df_test.shape}")
            print()


            ### Multivariate Imputation
            imputer = IterativeImputer(estimator = LinearRegression(), random_state=i)
            X_impute = imputer.fit_transform(df_train)
            df_train_imp = pd.DataFrame(data=X_impute, columns = df_train.columns)


            df_CV_imp = pd.DataFrame(data=imputer.transform(df_CV), columns = df_train.columns)
            df_test_imp = pd.DataFrame(data=imputer.transform(df_test), columns = df_train.columns)
            
            print('    *** Imputation Step Done *** ')


            ### Logistic Regression
            #logreg = LogisticRegression()

            #grid = GridSearchCV(logreg,                    # model
                       #param_grid = parameters,   # hyperparameters
                       #scoring='accuracy')

            #grid.fit(df_CV_imp, y_val, groups = groups_val, )


            train_score = np.zeros(len(ParameterGrid(param_grid)))
            val_score = np.zeros(len(ParameterGrid(param_grid)))
            models = []

            # loop through all combinations of hyperparameter combos
            for p in range(len(ParameterGrid(param_grid))):
                params = ParameterGrid(param_grid)[p]
                print('   ',params) 
                clf = LogisticRegression(**params, random_state = i, n_jobs=1) # initialize the classifier
                clf.fit(df_train_imp,y_train) # fit the model
                models.append(clf) # save it
                # calculate train and validation accuracy scores
                y_train_pred = clf.predict(df_train_imp)
                train_score[p] = accuracy_score(y_train,y_train_pred)
                y_val_pred = clf.predict(df_CV_imp)
                val_score[p] = accuracy_score(y_val,y_val_pred)
                print('   ',train_score[p],val_score[p])

            # print out model parameters that maximize validation accuracy
            print('best model parameters:',ParameterGrid(param_grid)[np.argmax(val_score)])
            print('corresponding validation score:',np.max(val_score))
            # collect and save the best model
            final_models.append(models[np.argmax(val_score)])
            # calculate and save the test score
            #y_test_pred = final_models[-1].predict(X_test_prep)
            #test_scores[i] = accuracy_score(y_test,y_test_pred)
            #print('test score:',test_scores[i])
"""